In [ ]:
import plotly.graph_objects as go
import numpy as np
from scipy.optimize import minimize, differential_evolution
import time
import pandas as pd

In [ ]:
# Целевая функция
def goldstein_price(x):
    x, y = x
    term1 = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2))
    term2 = (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return term1 * term2

In [ ]:
# Построение интерактивного графика
x = np.linspace(-2, 2, 400)
y = np.linspace(-2, 2, 400)
X, Y = np.meshgrid(x, y)
Z = np.array([goldstein_price([xi, yi]) for xi, yi in zip(np.ravel(X), np.ravel(Y))])
Z = Z.reshape(X.shape)

color = [
        [0, 'black'],
        [0.0001, 'purple'],
        [0.002, 'blue'],
        [0.005, 'cyan'],
        [0.03, 'green'],
        [0.1, 'yellow'],
        [0.3, 'orange'],
        [0.75, 'red'],
        [1, 'white'],
        ]

fig = go.Figure([go.Surface(z=Z, x=X, y=Y, colorscale=color)])
fig.update_layout(title='Функция Гольдштейна-Прайса',
                  autosize=True,
                  scene=dict(
                            xaxis_title='x1',
                            yaxis_title='x2',
                            zaxis_title='f(x1, x2)',
                            ),
                  width=800, height=800)

fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 100 прогонов
num_runs = 100
gradient_descent_results = []
differential_evolution_results = []
run_times = []
bounds = [(-2, 2), (-2, 2)]

# Метод градиентного спуска
for _ in range(num_runs):
    # Случайная начальная точка в пределах
    initial_point = np.random.uniform(-2, 2, size=2)
    result = minimize(goldstein_price, initial_point, method='BFGS')
    gradient_descent_results.append(result.fun)

# Алгоритм дифференциальной эволюции
for _ in range(num_runs):
    start_time = time.time()
    result = differential_evolution(goldstein_price, bounds, mutation=0.8,
                                    recombination=0.7, maxiter=1000, tol=1e-10)
    end_time = time.time()
    run_times.append(end_time - start_time)
    differential_evolution_results.append(result.fun)

In [ ]:
print(np.round(gradient_descent_results))
print(np.round(differential_evolution_results))

[ 84. 840.  30.  30.  30.  30.  30.  84.   3.  84.   3.  84.  30.   3.
   3.   3.   3.  84.   3.  30. 840.  30.  84.  30.   3.  30.   3.  30.
  30.  84.  84.   3.   3.   3.  30.   3.  30.   3.  30.  84.   3.  30.
   3.   3.   3.  84.  30.   3.   3.  84.  30.  30.   3.   3.  30.   3.
  30.   3.   3.   3.  30.   3.  84. 840.   3.   3.  84.   3.   3.  30.
   3.  30.  30.   3.   3.   3.  30.  84.   3. 840.   3.  84. 840.   3.
  30.   3.  30.  30.  84. 840.  30.   3.   3.   3.   3.  30.   3.  30.
 840.  30.]
[3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3.]


In [ ]:
print("Среднее значение градиентного спуска:" , np.mean(gradient_descent_results))
print("Дисперсия градиентного спуска:       " , np.var(gradient_descent_results))
print("Среднее дифференциальной эволюции:   " , np.mean(differential_evolution_results))
print("Дисперсия дифференциальной эволюции: " , np.var(differential_evolution_results))

Среднее значение градиентного спуска: 83.4600000000036
Дисперсия градиентного спуска:        43856.348400000104
Среднее дифференциальной эволюции:    3.0000000000004765
Дисперсия дифференциальной эволюции:  2.9719400592889836e-25


In [ ]:
target_value = 3.0
tolerance = 1e-100

start_time = time.time()
result = differential_evolution(goldstein_price, bounds, mutation=0.8,
                                recombination=0.7, maxiter=1000, tol=tolerance)

end_time = time.time()
run_time = end_time - start_time

# print(result.fun)
print("Время достижения глобального максимума:", run_time)

Время достижения глобального максимума: 4.41874623298645


In [ ]:
print("Среднее время:", np.mean(run_times))
print("Дисперсия времени:", np.var(run_times))

Среднее время: 0.1566551399230957
Дисперсия времени: 0.0012089174075573283


In [23]:
# Визуализация значений целевой функции по поколениям
class CallbackStore:
    def __init__(self):
        self.values = []

    def __call__(self, xk, convergence):
        self.values.append(goldstein_price(xk))

callback_store = CallbackStore()

bounds = [(-2, 2), (-2, 2)]
result = differential_evolution(goldstein_price, bounds, mutation=0.8,
                                recombination=0.7, maxiter=1000, tol=1e-10,
                                callback=callback_store, polish=False)

fig = go.Figure([go.Scatter(y=callback_store.values,
                            mode='lines+markers',
                            marker=dict(color='blue')
                            )
                ])

fig.update_layout(
    title="График изменения значений целевой функции по поколениям",
    xaxis_title="Поколение",
    yaxis_title="Значение функции",
)

fig.show()

In [ ]:
results_df = pd.DataFrame({
    "Метрика": ["Среднее значение", "Дисперсия"],
    "Время выполнения": [np.mean(run_times), np.var(run_times)],
    "Значение целевой функции": [np.mean(differential_evolution_results), np.var(differential_evolution_results)]
})

print(results_df)

            Метрика  Время выполнения  Значение целевой функции
0  Среднее значение          0.156655              3.000000e+00
1         Дисперсия          0.001209              2.971940e-25


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

data = load_iris()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Целевая функция для оптимизации
def objective_function(params, model_type):
    if model_type == 'svm':
        C, gamma = params
        model = SVC(C=C, gamma=gamma, random_state=42)
    elif model_type == 'knn':
        n_neighbors = int(params[0])
        model = KNeighborsClassifier(n_neighbors=n_neighbors)
    elif model_type == 'rf':
        n_estimators, max_depth = int(params[0]), int(params[1])
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)

    # Вычисление метрики кросс-валидации (accuracy)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    return -scores.mean()

# Границы параметров для каждого классификатора
param_bounds = {
    'svm': [(0.1, 100), (0.0001, 1)],  # C и gamma для SVM
    'knn': [(1, 30)],  # n_neighbors для KNN
    'rf': [(10, 200), (1, 20)]  # n_estimators и max_depth для Random Forest
}

def optimize_parameters(model_type):
    bounds = param_bounds[model_type]
    result = differential_evolution(objective_function, bounds, args=(model_type,), mutation=0.8, recombination=0.7, maxiter=50, tol=1e-3)
    return result

result_svm = optimize_parameters('svm')
result_knn = optimize_parameters('knn')
result_rf = optimize_parameters('rf')

In [ ]:
# Вывод оптимальных параметров
print("Оптимальные параметры для SVM:", result_svm.x)
print("Оптимальные параметры для KNN:", result_knn.x)
print("Оптимальные параметры для Random Forest:", result_rf.x)

Оптимальные параметры для SVM: [4.10942691e+01 3.28646948e-02]
Оптимальные параметры для KNN: [1.1673591]
Оптимальные параметры для Random Forest: [127.24739892  14.54259489]


In [ ]:
# Построение интерактивного графика
x = np.linspace(0.1, 100, 100)
y = np.linspace(0.0001, 1, 100)
X, Y = np.meshgrid(x, y)
Z = np.array([objective_function([xi, yi], 'svm') for xi, yi in zip(np.ravel(X), np.ravel(Y))])
Z = Z.reshape(X.shape)

fig = go.Figure([go.Surface(z=Z, x=X, y=Y, colorscale='rainbow')])
fig.update_layout(title='SVM',
                  autosize=True,
                  scene=dict(
                            xaxis_title='x1',
                            yaxis_title='x2',
                            zaxis_title='f(x1, x2)',
                            ),
                  width=800, height=800)

fig.show()